## Hazard assessment methodology
This notebook calculates return levels for extreme precipitation events based on annual maximum precipitation data.

In [ ]:
import numpy as np
import xarray as xr

from scipy.stats import gumbel_r

We read the NetCDF dataset containing daily precipitation data (tp) 

In [ ]:
# Load the dataset
ds_prec = xr.open_dataset('../data_conv/TOT_PREC_italy_UERRA.nc')
daily_precip = ds_prec['tp']

# Calculate the maximum one-day precipitation per year
annual_max_precip = daily_precip.resample(time='YE').max()
print(annual_max_precip)

We define Return Periods and Exceedance Probabilities: Return periods (e.g., 10, 20, 30, 50, 100, 150 years) are specified, and their corresponding exceedance probabilities are calculated. The exceedance probability represents the likelihood of surpassing a certain precipitation threshold in any given year

In [ ]:
return_periods = np.array([10, 20, 30, 50, 100, 150])
exceedance_probs = 1 - (1 / return_periods)

# Prepare a new dataset to store return levels for each period and each grid cell
return_levels_ds = xr.Dataset()

The notebook loops over every grid cell (longitude, and latitude), extracting the annual maximum precipitation values for each. For each grid cell the Gumbel distribution (gumbel_r.fit()) is fitted to the annual maxima series to determine location (loc) and scale (scale) parameters. The we calculate the return levels for the specified return periods using the previous parameters.

In [ ]:
# Loop over each grid cell (x, y) and fit the Weibull distribution
for x in annual_max_precip['x']:
    for y in annual_max_precip['y']:
        # Extract the annual maximum series for the current grid cell
        annual_max_values = annual_max_precip.sel(x=x, y=y).values
        annual_max_values = annual_max_values[~np.isnan(annual_max_values)]  # Remove NaNs

        if len(annual_max_values) > 0:
            # Fit the Gumbel distribution to the annual maxima for this grid cell
            loc, scale = gumbel_r.fit(annual_max_values)

            # Calculate the exceedance probabilities for the return periods
            exceedance_probs = 1 - (1 / np.array(return_periods))

            # Calculate return levels for the specified return periods using the Gumbel parameters
            return_levels = gumbel_r.ppf(exceedance_probs, loc, scale)


            # Store return levels in the dataset for each return period
            for rp, rl in zip(return_periods, return_levels):
                return_period_label = f"return_period_{rp}_y"
                if return_period_label not in return_levels_ds:
                    return_levels_ds[return_period_label] = xr.full_like(annual_max_precip.isel(time=0), np.nan)
                return_levels_ds[return_period_label].loc[{'x': x, 'y': y}] = rl

# Add coordinates and attributes to the new dataset
return_levels_ds['x'] = annual_max_precip['x']
return_levels_ds['y'] = annual_max_precip['y']
return_levels_ds.attrs['description'] = 'Return levels for specified return periods based on GEV distribution fit'

The "return_levels_ds" dataset represents the precipitation amount expected for each return period in each grid cell.

In [ ]:
# Save the return levels to a NetCDF file
return_levels_ds.to_netcdf('../indicators/uerra/return_levels_gumbel.nc')
return_levels_ds